The following tutorial shows how the anger words can be applied to identifying anger in general Twitter data. Here, we count tweets containing at least one of the anger words in data collected in San Diego during the 2018 State of the Union.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from cytoolz import concat
import re
import os

def anger_(l):
    x = [w for w in re.findall(r'\bliar\b|\bliars\b|\blying\b|\blies\b|\bhypocrite\b|\bhypocrites\b|\bhypocrisy\b|\bhypocritical\b|\basshole\b|\bassholes\b|\bbullshit\b|\bdisgrace\b|\bdisgraces\b|\bdisgraced\b|\bdisgraceful\b|\bstfu\b|\bdisgusting\b|\bdisgusted\b|\bdisgusts\b|\bscum\b|\binfuriate\b|\binfuriates\b|\binfuriating',l)]
    a = [w for w in re.findall(r'\bfuck you\b|\bthe fuck up\b|\bpiece of shit\b|\bgo fuck yourself\b',l)]
    b = [w for w in re.findall(r'\b(piss)\b.*off',l)]
    c = [w for w in re.findall(r'\boff\b.*\b(piss)\b',l)]
    d = [w for w in re.findall(r'\b(fuck)\b.*\boff\b',l)]
    e = [w for w in re.findall(r'\boff\b.*\b(fuck)\b',l)]
    z = []
    lists = [x,a,b,c,d,e,]
    for q in lists:
        for s in q:
            z.append(s)
    return z

#Count occurrences of anger words
def angertopic(filepath):
    df = pd.read_csv(filepath)
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False]
    df['anger words'] = df['TEXT'].str.lower().apply(anger_)
    words = list(concat(df['anger words']))
    wordcounts = Counter(words)
    if wordcounts == Counter():
        dic = {'Anger Words': 0, 'Tweet Count': len(df)}
        counts = pd.DataFrame.from_dict(dic,orient='index')
        counts.columns = ['Count']
    else:
        keywords = {'liar':0, 'liars':0,'lying':0, 'lies':0, 
                'hypocrite':0,'hypocrites':0, 'hypocrisy':0, 'hypocritical':0,
                'asshole':0, 'assholes':0, 'bullshit':0, 
                'fuck':0, 'fuck you':0, 
                'disgrace':0, 'disgraces':0, 'disgraced':0, 'disgraceful':0, 
                'piece of shit':0, 'the fuck up':0, 
                'piss':0, 'stfu':0,
                'disgusting':0, 'disgusted':0, 'disgusts':0, 
                'go fuck yourself':0, 'scum':0, 
                'infuriate':0, 'infuriates':0, 'infuriating':0, 'infuriated':0}
        kw = pd.DataFrame.from_dict(keywords, orient='index')
        wc = pd.DataFrame.from_dict(wordcounts, orient='index')
        counts = kw.join(wc, how='left', lsuffix = 'kw', rsuffix = 'wc').fillna(0)
        counts['Counts'] = counts['0kw'] + counts['0wc']
        del counts['0kw']
        del counts['0wc']
        if 'fuck' in counts.index:
            counts.index.set_value(counts.index, 'fuck', 'fuck off')
        if 'piss' in counts.index:
            counts.index.set_value(counts.index, 'piss', 'piss(ed) off')
        counts.loc['~Total'] = counts.sum()
        counts.loc['~Tweet Count'] = len(df)
        counts['Percent'] = (counts['Counts']/len(df))*100
    print(counts.sort_index(axis=0, ascending=True))

def angertopics(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            angertopic(os.path.join(directory, filename))
            continue
        else:
            continue

The next cell shows what the data looks like. The only required fields for this code are the TEXT column, which has one tweet per cell. Note that this is case sensitive, so the column must be called TEXT (not 'text' or 'Text').

We recommend filtering data to English tweets only and removing retweets, which can be done by adding to the available code or by preprocessing data.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/HDMA-SDSU/HDMA-Linguistic/master/Anger_Words/Data/2018%20State%20of%20the%20Union%20cut.csv')
df.head()

,GEOCODING TYPE,TWEET_ID,KEYWORD,CITY,CREATED_AT,CREATED_AT_LOCAL,TEXT,FROM_USER,FROM_USER_NAME,LOCATION,...,TO_USER_NAME,IN_REPLY_TO_STATUS_ID,SOURCE,LANGUAGE,IS_VALID_ONLY_FOR_FLU,GEOCODE_ENGINE,GEOCODE_TYPE,PROFILE_GEO,PROFILE_LEVEL,FINAL_GEO
0,Location,9.600000e+17,sotu,San_Diego,2/3/18 7:57,2/2/18 23:57,@RepMattGaetz This from a guy who has been arr...,CounterSocial66,Teddy Pendergrass,"San Diego, CA",...,NaN,NaN,Twitter for iPad,en,False,HDMA,userProfile,"-116.77021,33.0282",undefined,"-116.77021, 33.0282"
1,Location,9.600000e+17,state of the union,San_Diego,2/3/18 7:27,2/2/18 23:27,A new poll from NBC News released on Thursday ...,Enciniman1,James Parr,"Encinitas, CA",...,NaN,NaN,Twitter for Android,en,False,HDMA,userProfile,"-117.29198,33.03699",undefined,"-117.29198, 33.03699"
2,Location,9.600000e+17,sotu,San_Diego,2/3/18 7:17,2/2/18 23:17,@realDonaldTrump Wow... The idiot he retweeted...,MFooteOBSDbeer,Michelle Foote,"Ocean Beach, San Diego, CA",...,NaN,NaN,Twitter for Android,en,False,NaN,NaN,NaN,NaN,NaN
3,Place,9.600000e+17,sotu,San_Diego,2/3/18 5:41,2/2/18 21:41,@tedlieu @realDonaldTrump @FBI I didn't read e...,ZangerJames,James Zanger,"San Diego, CA",...,NaN,NaN,Twitter for Android,en,False,NaN,NaN,NaN,NaN,"-117.10498915,32.8100122"
4,Location,9.600000e+17,state of the union,San_Diego,2/3/18 4:31,2/2/18 20:31,Nancy Pelosi As a citizen of the State of Cal...,KathyCazin,Kathy Cazin,"San Diego, CA",...,NaN,NaN,Twitter Web Client,en,False,HDMA,userProfile,"-116.77021,33.0282",undefined,"-116.77021, 33.0282"


Here, the anger check code is applied using the angertopic function. In this tutorial, data is read from a URL. When you apply this to your own data using the Gener-Anger-Check notebook (https://github.com/HDMA-SDSU/HDMA-Linguistic/blob/master/Anger_Words/General-Anger-Check.ipynb), data will be loaded from your computer by typing the path to the file on your computer, e.g. angertopic('/Users/ilana/Anger/data.xlsx')

For the tutorial, the results are printed in this notebook. When using the General-Anger-Check notebook, data will be exported to your computer as a CSV file in the same format shown here. The resulting file will be called [original file name] anger.csv.

In [3]:
angertopic('https://raw.githubusercontent.com/HDMA-SDSU/HDMA-Linguistic/master/Anger_Words/Data/2018%20State%20of%20the%20Union%20cut.csv')

                  Counts     Percent
asshole              0.0    0.000000
assholes             0.0    0.000000
bullshit             5.0    0.333333
disgrace             6.0    0.400000
disgraced            1.0    0.066667
disgraceful          8.0    0.533333
disgraces            0.0    0.000000
disgusted            4.0    0.266667
disgusting           8.0    0.533333
disgusts             0.0    0.000000
fuck off             0.0    0.000000
fuck you             1.0    0.066667
go fuck yourself     0.0    0.000000
hypocrisy            0.0    0.000000
hypocrite            0.0    0.000000
hypocrites           0.0    0.000000
hypocritical         0.0    0.000000
infuriate            0.0    0.000000
infuriated           0.0    0.000000
infuriates           0.0    0.000000
infuriating          0.0    0.000000
liar                16.0    1.066667
liars                0.0    0.000000
lies                22.0    1.466667
lying               15.0    1.000000
piece of shit        0.0    0.000000
p

The first column shows all of the anger words, as well as the total number of anger words (~Total) and the total number of tweets (~Tweet Count). The second column shows the number of occurrences of each keyword, and the third column shows the anger word occurrences as a percentage of the total number of tweets. Note that these are percentages out of 100%, not decimals (i.e. 0.27 means 0.27%, not 27%).

See the Anger Words Technical Document (https://github.com/HDMA-SDSU/HDMA-Linguistic/blob/master/Anger_Words/Anger-Words-Technical-Doc.pdf) for more detailed explanation and instructions.